In [ ]:
from google.colab import files

jsonfile = files.upload()

Saving microbiology.jsonl to microbiology.jsonl


In [ ]:
! pip install transformers[torch]
! pip install accelerate -U
! pip install scikit-learn
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
import json

def parse_jsonl(jsonl_file_path,**kwargs):
    file = open(jsonl_file_path, 'r')
    lines = file.readlines()
    file.close()
    field = "None"
    filter = "None"
    value_to_retain = "None"
    key_for_value = "None"
    list_of_dicts = []
    for key, value in kwargs.items():
        if key == "field":
            field = value
        elif key == "filt":
            filter = value
        elif key == "retain_value":
            value_to_retain = value
        elif key == "key_for_value":
            key_for_value = value
        else:
            raise ValueError("Allowed kwargs are field and filter")
    if field=="None" and filter=="None":
        for line in lines:
            json_object = json.loads(line)
            if value_to_retain == "None" and key_for_value == "None":
              list_of_dicts.append(json_object)
            elif (value_to_retain == "None" and key_for_value != "None") or (value_to_retain != "None" and key_for_value == "None"):
              raise ValueError("You cannot set value to retain without its key or viceversa")
            else:
              list_of_dicts.append({key_for_value: json_object[value_to_retain]})
        return list_of_dicts
    elif (field=="None" and filter!="None") or (field!="None" and filter=="None"):
        raise ValueError("You cannot set filter without field or field without filter")
    else:
        for line in lines:
            json_object = json.loads(line)
            if json_object[field] == filter:
                if value_to_retain == "None" and key_for_value == "None":
                  list_of_dicts.append(json_object)
                elif (value_to_retain == "None" and key_for_value != "None") or (value_to_retain != "None" and key_for_value == "None"):
                  raise ValueError("You cannot set value to retain without its key or viceversa")
                else:
                  list_of_dicts.append({key_for_value: json_object[value_to_retain]})
            else:
                continue
        return list_of_dicts

data = parse_jsonl("microbiology.jsonl", retain_value="message_2", key_for_value="text")
data[0]

{'text': "Changes in gut microbiota composition can significantly affect the immune system of animals and their susceptibility to infectious diseases. The gut microbiota, which consists of trillions of microorganisms, including bacteria, fungi, and viruses, plays a crucial role in maintaining the overall health of the host organism. These microorganisms contribute to various physiological processes, such as digestion, metabolism, and immune system regulation.\n\nThe immune system and gut microbiota have a bidirectional relationship, where the immune system shapes the composition of the gut microbiota, and the gut microbiota influences the development and function of the immune system. Here are some ways in which changes in gut microbiota composition can affect the immune system and susceptibility to infectious diseases:\n\n1. Barrier function: A healthy gut microbiota helps maintain the integrity of the gut barrier, which prevents the entry of pathogens and harmful substances into the 

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Split the dataset into training and validation sets
train_dataset, validation_dataset = train_test_split(
    data,
    test_size=0.2,  # You can adjust the test_size as needed
    random_state=42,
)

train_dataset = Dataset.from_list(train_dataset)
validation_dataset = Dataset.from_list(validation_dataset)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/640 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/640 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-microbiology",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 